In [1]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 8.6 MB/s eta 0:00:00a 0:00:01
  Using cached pillow-11.1.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 21.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 24.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 22.4 MB/s eta 0:00:0000:0100:01
Using cached pillow-11.1.0-cp313-cp313-macosx_11_0_arm64.whl (3.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 19.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 23.0 MB/s e

In [4]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [5]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [49]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Create source embeddings for the text column

In [7]:
from sentence_transformers import SentenceTransformer

/Users/dineshchandgeetharavichandran/Desktop/Dinesh/Project/Data-Science/GenAI/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [10]:
vectors

array([[-0.00247391,  0.03626726, -0.05290464, ..., -0.09152355,
        -0.03970001, -0.04330486],
       [-0.03357266,  0.00980519, -0.03250133, ..., -0.05165476,
         0.02245886, -0.03156189],
       [-0.01865333, -0.04051303, -0.01235392, ...,  0.00610584,
        -0.07179647,  0.02773847],
       ...,
       [-0.0006646 ,  0.04252121, -0.05645504, ...,  0.01315468,
        -0.0318356 , -0.04357661],
       [-0.03317154,  0.03252462, -0.0248484 , ...,  0.0117442 ,
         0.05747124,  0.00571026],
       [-0.00166392,  0.00413833, -0.04597079, ...,  0.02008527,
         0.05656249, -0.00161594]], shape=(8, 768), dtype=float32)

In [9]:
vectors.shape

(8, 768)

In [12]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [13]:
import faiss

index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [14]:
index.add(vectors)

In [15]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x152d94f60> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [26]:
#search_query = "I want to buy a polo t-shirt"
search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [27]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [28]:
faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [29]:
distances, I = index.search(svec, k=2)
distances

array([[0.9273728, 1.1601744]], dtype=float32)

In [30]:
I

array([[6, 7]])

In [31]:
I.tolist()

[[6, 7]]

In [32]:
row_indices = I.tolist()[0]
row_indices

[6, 7]

In [33]:
df.loc[row_indices]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [25]:
search_query

'I want to buy a polo t-shirt'

You can see that the two results from the dataframe are similar to a search_query